# Fetch Data

This project relies on [explain here: data sources it uses]

Explain basic methodology:
- functions all live as a package under scripts
- actual concrete processing steps done one-by-one in this notebook
- then once it's done, turn it into a function that goes in the package
- keep the notebook very high level

## Fetching SAM UEI data

Read in functions from `fetch_firm_ueis` submodule.

In [1]:
from scripts.data_fetchers.fetch_firm_ueis import read_sam_json, fetch_sam_data, square_and_clean_SAM_JSON

2024-12-15 13:30:11.477 | INFO     | scripts.config:<module>:11 - PROJ_ROOT path is: /Users/jaspercooper/Dropbox/01_Projects/01_python_projects/predict_marketing_success


This function sends a query to the SAM entity management API that gets all currently active 8(a) firms. Because 8(a) firms are, by definition, in the US and small businesses, a lot of the filtering is taken care of simply by specifying that they are 8(a)-certified, so the request is quite minimal.

This function outputs a JSON file into the raw data folder.

### FIX: API query currently only pulling 10 records
Need to figure out how to get API bulk-downloading the csv


In [7]:
fetch_sam_data()

{'error': {'code': 'OVER_RATE_LIMIT', 'message': 'The API key has exceeded the rate limits.'}}


This function reads in the JSON data downloaded in the previous step

In [3]:
data = read_sam_json()

JSON has the following keys: totalRecords, entityData, links


This function reformats the data as a pandas dataframe and cleans out the lengthy prefixes from the column names.

In [4]:
df = square_and_clean_SAM_JSON(data)

Extracted entity data from JSON
Flattened JSON to df
Created a dataset with dimensions (10, 131) and the following keys: samRegistered, ueiSAM, entityEFTIndicator, cageCode, dodaac, legalBusinessName, dbaName, purposeOfRegistrationCode, purposeOfRegistrationDesc, registrationStatus, registrationDate, lastUpdateDate, registrationExpirationDate, activationDate, ueiStatus, ueiExpirationDate, ueiCreationDate, noPublicDisplayFlag, exclusionStatusFlag, exclusionURL, dnbOpenData, entityURL, entityDivisionName, entityDivisionNumber, entityStartDate, fiscalYearEndCloseDate, submissionDate, addressLine1, addressLine2, city, stateOrProvinceCode, zipCode, zipCodePlus4, countryCode, addressLine1, addressLine2, city, stateOrProvinceCode, zipCode, zipCodePlus4, countryCode, congressionalDistrict, entityStructureCode, entityStructureDesc, entityTypeCode, entityTypeDesc, profitStructureCode, profitStructureDesc, organizationStructureCode, organizationStructureDesc, stateOfIncorporationCode, stateOfInco

In [5]:
df.head()

,samRegistered,ueiSAM,entityEFTIndicator,cageCode,dodaac,legalBusinessName,dbaName,purposeOfRegistrationCode,purposeOfRegistrationDesc,registrationStatus,...,middleInitial,lastName,title,addressLine1,addressLine2,city,stateOrProvinceCode,zipCode,zipCodePlus4,countryCode
0,Yes,TKB9GELBZGG4,None,9YNT8,None,"CASBRO DEVELOPMENT, LLC",None,Z2,All Awards,Active,...,None,None,None,None,None,None,None,None,None,None
1,Yes,XTDKG6B4XL11,None,9YCT5,None,CHOCTAW ADVANTAGE SOLUTIONS LLC,None,Z2,All Awards,Active,...,None,Kost,President,435 E Sandstone RD,STE 100 N,Durant,OK,74701,None,USA
2,Yes,XTT4DAG5BEN5,None,9YDQ7,None,CHOCTAW MISSION OPERATIONS LLC,None,Z2,All Awards,Active,...,None,Kern,President,435 E Sandstone RD,STE 100 M,Durant,OK,74701,None,USA
3,Yes,VQ3VALRQTYK5,None,9YCM3,None,"MIDNIGHT SUN FEDERAL CONSTRUCTION, LLC",None,Z2,All Awards,Active,...,None,None,None,None,None,None,None,None,None,None
4,Yes,PTR8P6WK3F74,None,9XV39,None,"ADVANCED CHENEGA SERVICES, LLC",None,Z2,All Awards,Active,...,None,Rankin,None,"1 E. Uwchlan Avenue, Suite 407",None,Exton,PA,19341,None,USA


Output the data to csv format


In [6]:
from scripts.config import INTERIM_DATA_DIR
df.to_csv(f'{INTERIM_DATA_DIR}/sam_entity_data.csv', index=False)

# Fetch USAspending data on awards to 8(a) firms

This is the code to use once the API request issue above is resolved:

In [5]:
# import pandas as pd
# from scripts.config import INTERIM_DATA_DIR
# df = pd.read_csv(f'{INTERIM_DATA_DIR}/sam_entity_data.csv', usecols= ["ueiSAM"])

Pull in data on UEIs of 8(a)s from SAM.gov

In [1]:
import pandas as pd
from scripts.config import RAW_DATA_DIR
df = pd.read_csv(f'{RAW_DATA_DIR}/raw_uei_data/EntityRegistration-20241215-014119.csv', usecols= ["Unique Entity ID"])
df.columns = ["uei"]

2024-12-15 15:52:12.312 | INFO     | scripts.config:<module>:11 - PROJ_ROOT path is: /Users/jaspercooper/Dropbox/01_Projects/01_python_projects/predict_marketing_success


Inspect dataframe

In [14]:
df.shape

(5000, 1)

Grab USAspending data on spending with those firms for past 3 years. The function `fetch_awards()` looks on the API to see whether it can find the info for a given firm using its UEI. When it succesfully retrieves a record, it stores it in a big list of JSON entries.

In [2]:
from scripts.data_fetchers.fetch_award_data import fetch_awards

# List of UEIs to query
ueis = df['uei'].tolist()

fetch_awards(ueis = ueis, year = 2022)
fetch_awards(ueis = ueis, year = 2023)
fetch_awards(ueis = ueis, year = 2024)


Collected 19 successful responses and 1 unsuccessful responses.
successful_responses saved to /Users/jaspercooper/Dropbox/01_Projects/01_python_projects/predict_marketing_success/data/raw/raw_award_data/awards_to_8a_2022.json
Collected 19 successful responses and 1 unsuccessful responses.
successful_responses saved to /Users/jaspercooper/Dropbox/01_Projects/01_python_projects/predict_marketing_success/data/raw/raw_award_data/awards_to_8a_2023.json


The `concatenate_json_files()` function reads in all the files created in the previous script, then loops through them, turning them into dataframes with the year attached as a variable.

In [1]:
from scripts.config import RAW_DATA_DIR
from scripts.data_fetchers.fetch_award_data import concatenate_json_files
all_awards = concatenate_json_files(input_dir = f"{RAW_DATA_DIR}/raw_award_data")

2024-12-15 16:19:16.170 | INFO     | scripts.config:<module>:11 - PROJ_ROOT path is: /Users/jaspercooper/Dropbox/01_Projects/01_python_projects/predict_marketing_success


Inspect concatenated data

In [2]:
all_awards.head()

,recipient_id,name,duns,uei,amount,state_province,year
0,775b7921-5e61-1b97-10bf-6632dd994608-C,"EMERGING CONSULTING, LLC",080117665,E2N5TXXYYYP7,20000.00,VA,2022
1,d7bef74d-1611-a8fd-ec9c-70668d28d63a-C,KTL COMMUNICATIONS LLC,079198829,HHANSGF1ELX3,298255.00,VA,2022
2,bf277e2b-45d6-1b24-38bb-c1790ccda0e1-C,"H & L ENVIRONMENTAL SERVICES, LLC",964426451,JRRRE11LEM21,1737906.38,NC,2022
3,f0974e90-813c-bb5a-74b1-59c0fcf8fbda-C,GSI NORTH AMERICA INC,081267392,K7ZHNF6YJMB3,2894705.00,HI,2022
4,d0241de6-18ac-7596-08f2-c90c556c36b9-C,GSI NORTH AMERICA INC,079386548,EHM2NWLJHSJ7,0.00,NJ,2022


Output the data as a csv to the interim data folder

In [3]:
from scripts.config import INTERIM_DATA_DIR
all_awards.to_csv(f'{INTERIM_DATA_DIR}/award_data.csv', index=False)